# **Customer Churn Prediction**

#### **What is customer churn?**  
Customer churn refers to the percentage of customers who stop using a company's product or service within a given time frame. This metric helps businesses gauge customer satisfaction and loyalty while also providing insights into potential revenue fluctuations.

Churn is especially critical for subscription-based businesses, such as SaaS companies, which rely on recurring revenue. Understanding churn patterns allows them to anticipate financial impact and take proactive measures.

Also known as customer attrition, churn is the opposite of customer retention, which focuses on maintaining long-term customer relationships. Reducing churn should be a key part of any customer engagement strategy, ensuring consistent interactions between businesses and their customers, whether online or in person.

A strong customer retention plan plays a crucial role in minimizing churn. Companies should track churn rates regularly to assess their risk of revenue loss and identify areas for improvement.

<br>

**Source:** IBM. Customer Churn. Retrieved from https://www.ibm.com/think/topics/customer-churn

---

Prepare the libraries.

In [328]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from sklearn.metrics import classification_report, roc_auc_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from xgboost import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

Prepare the dataset/s.

In [329]:
pd.pandas.set_option('display.max_columns',None)

train_path = '../data/CustomerChurnDataset/customer_churn_dataset-testing-master.csv'
test_path = '../data/CustomerChurnDataset/customer_churn_dataset-training-master.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

Since we saw in our EDA that we only have 1 row of missing value/s from test_df lets drop that row.  
Prepare the train and test datasets.

In [330]:
test_df.dropna(inplace=True)

y_train = train_df['Churn']
x_train = train_df.drop(columns=['Churn'])

y_test = test_df['Churn']
x_test = test_df.drop(columns=['Churn'])

Get the numerical and categorical features.

In [331]:
categorical_features = x_train.select_dtypes(include=['object']).columns.tolist()
numerical_features = x_train.select_dtypes(include=['int64','float64']).columns.tolist()

print(categorical_features)
print(numerical_features)

['Gender', 'Subscription Type', 'Contract Length']
['CustomerID', 'Age', 'Tenure', 'Usage Frequency', 'Support Calls', 'Payment Delay', 'Total Spend', 'Last Interaction']


Prepare the preprocessor for the pipeline.

In [332]:
preprocessor = ColumnTransformer([
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
], remainder='passthrough')

Set up the hyperparameters for tuning.

In [333]:
params_space = {
    'eta': hp.uniform('eta', 0.01,0.2),
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform ('gamma', 1,9),
    'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
    'reg_lambda' : hp.uniform('reg_lambda', 0,1),
    'subsample': hp.uniform('subsample', 0.5,1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 50, 1000, 10),
    'seed': 0
}

def objective(params):
    # convert the quniforms into int since they are float
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params['min_child_weight'] = int(params['min_child_weight'])
    params['reg_alpha'] = int(params['reg_alpha'])
    
    pipeline = Pipeline([
        ('preprocessing', preprocessor),
        ('classifier', XGBClassifier(**params, use_label_encoder=False, eval_metric="logloss"))
    ])

    pipeline.fit(x_train, y_train)
    y_pred = pipeline.predict(x_test)
    auc = roc_auc_score(y_test, y_pred)
    print(f"Params: {params}, AUC-ROC: {auc:.4f}")

    return {'loss': -auc, 'status': STATUS_OK}

Run the hyperparameter tuning.


In [ ]:
trials = Trials()

best_params = fmin(
    fn = objective,
    space = params_space,
    algo = tpe.suggest,
    max_evals = 100,
    trials = trials
)

print("\n✅ Best Hyperparameters Found:", best_params)

Get the best hyperparameters.


In [ ]:
int_params = {"max_depth", "n_estimators", "min_child_weight", "reg_alpha"}

def convert_params(params):
    return {k: int(v) if k in int_params else float(v) for k, v in params.items()}

best_params = convert_params(best_params)

Create the pipeline.

In [342]:
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', XGBClassifier(**best_params))
])

Train the model and make predictions.

In [343]:
pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)

Evaluate the model.

In [344]:
def evaluate_model(test, pred):
    accuracy = accuracy_score(test, pred)

    print(f"Final Model Accuracy: {accuracy:.4f}")
    print("XGBoost Classification Report:\n", classification_report(test, pred))
    print("XGBoost AUC-ROC:", roc_auc_score(test, pred))
    
evaluate_model(y_test, y_pred)

Final Model Accuracy: 0.7422
XGBoost Classification Report:
               precision    recall  f1-score   support

         0.0       0.70      0.70      0.70    190833
         1.0       0.77      0.78      0.77    249999

    accuracy                           0.74    440832
   macro avg       0.74      0.74      0.74    440832
weighted avg       0.74      0.74      0.74    440832

XGBoost AUC-ROC: 0.7369742139671895
